In [1]:
#| default_exp format
%load_ext autoreload
%autoreload 2

In [2]:
#| hide
import sys
from pathlib import Path
# Insert in Path Project Directory
sys.path.insert(0, str(Path().cwd().parent))

In [3]:
#| export
import re
from typing import Iterable, Tuple, Union

import pandas as pd
import numpy as np
from fastcore.utils import listify
from fastcore.xtras import Path
from geopy.distance import geodesic
from rich.progress import Progress
from pyarrow import ArrowInvalid

from extracao.constants import BW, BW_pattern

RE_BW = re.compile(BW_pattern)

MAX_DIST = 10  # Km

In [ ]:
#| export
def _read_df(folder: Union[str, Path], stem: str) -> pd.DataFrame:
    """Lê o dataframe formado por folder / stem.[parquet.gzip | fth | xslx]"""
    file = Path(f"{folder}/{stem}.parquet.gzip")
    try:
        df = pd.read_parquet(file)
    except (ArrowInvalid, FileNotFoundError) as e:
        raise e(f"Error when reading {file}")
    return df


# Formatação

> Este módulo possui funções auxiliares de formatação dos dados das várias fontes.


In [4]:
#| export
def parse_bw(
    bw: str,  # Designação de Emissão (Largura + Classe) codificada como string
) -> Tuple[str, str]:  # Largura e Classe de Emissão
    """Parse the bandwidth string"""
    if match := re.match(RE_BW, bw):
        multiplier = BW[match.group(2)]
        if mantissa := match.group(3):
            number = float(f"{match.group(1)}.{mantissa}")
        else:
            number = float(match.group(1))
        classe = match.group(4)
        return str(multiplier * number), str(classe)
    return "-1", "-1"


### Mesclagem
Função auxiliar para mesclar registros que são iguais das diversas bases, i.e. estão a uma distância menor que `MAX_DIST` e verificar a validade da mesclagem

In [22]:
#| export
def merge_close_rows(df_left, df_right):
    """Mescla os registros dos DataFrames `df_left` e `df_right` que estão a uma distância menor que MAX_DIST"""
    df1 = df_left.copy().reset_index(drop=True)
    df2 = df_right.copy().reset_index(drop=True)
    columns = ["Frequency", "Latitude", "Longitude"]
    for c in columns:
        df1[c] = df1[c].astype("float")
        df2[c] = df2[c].astype("float")
    df1.sort_values(columns, inplace=True)
    df2.sort_values(columns, inplace=True)
    with Progress(transient=True, refresh_per_second=2) as progress:
        task_left = progress.add_task(
            "[red]Iterando Tabela Principal...", total=len(df1)
        )
        for left in df1.itertuples():
            for right in df2[np.isclose(df2.Frequency, left.Frequency)].itertuples():
                if (
                    geodesic(
                        (left.Latitude, left.Longitude),
                        (right.Latitude, right.Longitude),
                    ).km
                    <= MAX_DIST
                ):
                    df1.loc[
                        left.Index, "Description"
                    ] = f"{left.Description} | {right.Description}"
                    df2 = df2.drop(right.Index)
                    break
            progress.update(
                task_left,
                advance=1,
                description=f"[green] Comparando Frequências {left.Frequency}MHz",
            )
        return pd.concat([df1, df2], ignore_index=True)


In [8]:
from extracao.reading import *
from extracao.main import format_matlab
from dotenv import load_dotenv

load_dotenv()

folder = Path.cwd().parent / 'dados'


In [9]:
base = format_matlab(read_base(folder))

In [10]:
base

,Id,Frequency,Latitude,Longitude,Description,Service,Station,Class,BW
0,#1,0.028,-22.662779,-43.476391,"[MOS] L, OP, Furnas Centrais Eletricas S A (01...",19,1557670,J9E,8.0
1,#2,0.030,-23.709999,-46.273335,"[MOS] L, OP, Furnas Centrais Eletricas S A (01...",19,1558412,J3E,2.0
2,#3,0.030,-23.441668,-46.590832,"[MOS] L, OP, Furnas Centrais Eletricas S A (01...",19,1557823,J3E,1.0
3,#4,0.030,-22.926666,-43.264999,"[MOS] L, OP, Furnas Centrais Eletricas S A (01...",19,859761,J3E,0.5
4,#5,0.030,-22.774166,-47.004444,"[MOS] L, OP, Furnas Centrais Eletricas S A (01...",19,859753,J3E,1.0
...,...,...,...,...,...,...,...,...,...
832361,#832362,148650.000,-2.509556,-44.294556,"[MOS] L, FB, Apil Seguranca E Vigilancia Ltda ...",19,1011671368,F1D,7.6
832362,#832363,4096125.000,-29.872139,-51.170582,"[MOS] L, FX, Petroleo Brasileiro S/A Petrobras...",19,698585682,F7D,25.0
832363,#832364,4265625.000,-29.876944,-51.170692,"[MOS] L, FX, Petroleo Brasileiro S/A Petrobras...",19,698585704,F7D,25.0
832364,#832365,4265625.000,-29.870222,-51.201305,"[MOS] L, FX, Petroleo Brasileiro S/A Petrobras...",19,698801288,F7D,25.0


In [11]:
aero = read_aero(folder)

In [24]:
df = merge_close_rows(base.iloc[-100:], aero)

Output()

In [15]:
df = pd.merge(base, aero, on=['Frequency'], how='outer')

In [16]:
def distancia(row):
    return geodesic((row[0], row[1]), (row[2], row[3])).km

In [17]:
condition = np.all(df[['Latitude_x', 'Longitude_x', 'Latitude_y', 'Longitude_y']].notna(), axis=1)

In [18]:
df.loc[condition, 'dist'] = df.loc[condition, ['Latitude_x', 'Longitude_x', 'Latitude_y', 'Longitude_y']].apply(distancia , axis=1)

In [19]:
condition = (df.dist.notna()) & (df.dist <= MAX_DIST) 
df.loc[condition, 'Description_x'] = df.loc[condition, 'Description_x'] + ' | ' + df.loc[condition, 'Description_y']  

In [20]:
df.shape

(884139, 22)

In [21]:
df.to_excel(folder / 'mesclados_outer_join.xlsx', index=False)

In [28]:
aero.to_excel(folder / 'aero.xlsx', index=False)

## Otimização dos Tipos de dados
A serem criados dataframes, normalmente a tipo de data é aquele com maior resolução possível, nem sempre isso é necessário, os arquivos de espectro mesmo possuem somente uma casa decimal, portanto um `float16` já é suficiente para armazená-los. As funções a seguir fazem essa otimização

Code below borrowed from https://medium.com/bigdatarepublic/advanced-pandas-optimize-speed-and-memory-a654b53be6c2

In [ ]:
#| export
def optimize_floats(
    df: pd.DataFrame,  # DataFrame a ser otimizado
    exclude: Iterable[str] = None,  # Colunas a serem excluidas da otimização
) -> pd.DataFrame:  # DataFrame com as colunas do tipo `float` otimizadas
    """Otimiza os floats do dataframe para reduzir o uso de memória"""
    floats = df.select_dtypes(include=["float64"]).columns.tolist()
    floats = [c for c in floats if c not in listify(exclude)]
    df[floats] = df[floats].apply(pd.to_numeric, downcast="float")
    return df


In [ ]:
#| export
def optimize_ints(
    df: pd.DataFrame,  # Dataframe a ser otimizado
    exclude: Iterable[str] = None,  # Colunas a serem excluidas da otimização
) -> pd.DataFrame:  # DataFrame com as colunas do tipo `int` otimizadas
    """Otimiza os ints do dataframe para reduzir o uso de memória"""
    ints = df.select_dtypes(include=["int64"]).columns.tolist()
    ints = [c for c in ints if c not in listify(exclude)]
    df[ints] = df[ints].apply(pd.to_numeric, downcast="integer")
    return df


In [ ]:
#| export
def optimize_objects(
    df: pd.DataFrame,  # DataFrame a ser otimizado
    datetime_features: Iterable[
        str
    ] = None,  # Colunas que serão convertidas para datetime
    exclude: Iterable[str] = None,  # Colunas que não serão convertidas
) -> pd.DataFrame:  # DataFrame com as colunas do tipo `object` otimizadas
    """Otimiza as colunas do tipo `object` no DataFrame para `category` ou `string` para reduzir a memória e tamanho de arquivo"""
    exclude = listify(exclude)
    datetime_features = listify(datetime_features)
    for col in df.select_dtypes(
        include=["object", "string", "category"]
    ).columns.tolist():
        if col not in datetime_features:
            if col in exclude:
                continue
            num_unique_values = len(df[col].unique())
            num_total_values = len(df[col])
            if float(num_unique_values) / num_total_values < 0.5:
                dtype = "category"
            else:
                dtype = "string"
            df[col] = df[col].astype(dtype)
        else:
            df[col] = pd.to_datetime(df[col]).dt.date
    return df


In [ ]:
#| export
def df_optimize(
    df: pd.DataFrame,  # DataFrame a ser otimizado
    datetime_features: Iterable[
        str
    ] = None,  # Colunas que serão convertidas para datetime
    exclude: Iterable[str] = None,  # Colunas que não serão convertidas
) -> pd.DataFrame:  # DataFrame com as colunas com tipos de dados otimizados
    """Função que encapsula as anteriores para otimizar os tipos de dados e reduzir o tamanho do arquivo e uso de memória"""
    if datetime_features is None:
        datetime_features = []
    return optimize_floats(
        optimize_ints(optimize_objects(df, datetime_features, exclude), exclude),
        exclude,
    )